"""
Module Description:
-------------------
Class to extract skills from text and align them to existing taxonomy

Ownership:
----------
Project: Leveraging Artificial intelligence for Skills Extraction and Research (LAiSER)
Owner:  George Washington University Institute of Public Policy
        Program on Skills, Credentials and Workforce Policy
        Media and Public Affairs Building
        805 21st Street NW
        Washington, DC 20052
        PSCWP@gwu.edu
        https://gwipp.gwu.edu/program-skills-credentials-workforce-policy-pscwp

License:
--------
Copyright 2024 George Washington University Institute of Public Policy

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated
documentation files (the “Software”), to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software,
and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the
Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR
OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Revision History:
-----------------
| Rev No. | Date | Author | Description |
|---------|------|--------|-------------|
| [1.0.0] | 06/05/2024 |      Satya Phanindra K. |  Created a standalone notebook for skill extraction
| [1.0.1] | 06/11/2024 |      Satya Phanindra K. |  Added GPU support for processing
| [1.0.1] | 06/20/2024 |      Satya Phanindra K. |  Added error handling and logging
| [1.0.2] | 07/01/2024 |      Satya Phanindra K. |  Threshold update for similarity and AI model
| [1.0.3] | 07/10/2024 |      Satya Phanindra K. |  Added seperate functions set for LLM usecases
| [1.0.4] | 07/13/2024 |      Satya Phanindra K. |  Add descriptions to each method
| [1.0.5] | 07/18/2024 |      Satya Phanindra K. |  Added CONDITIONAL GPU support for LLM
| [1.0.6] | 07/22/2024 |      Satya Phanindra K. |  Added support for SkillNer model for skill extraction, if GPU not available
| [1.0.7] | 07/25/2024 |      Satya Phanindra K. |  Calculate cosine similarities in bulk for optimal performance.
| [1.0.8] | 07/28/2024 |      Satya Phanindra K. |  Error handling for empty list outputs from extract_raw function

## Installing required packages

In [ ]:
!wget https://raw.githubusercontent.com/LAiSER-Software/extract-module/refs/heads/main/requirements.txt

In [ ]:
!pip3 install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import essential packages and libraries

In [ ]:
!python -m spacy download en_core_web_lg

In [3]:
# native packages
import sys
import os

# installed packages
import time
import spacy
import torch
import pandas as pd
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from sklearn.metrics.pairwise import cosine_similarity
from skillNer.skill_extractor_class import SkillExtractor
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

## Global variables and Function Definitions from internal packages

#### Utils

- The content in the code cell below are copied from the `../laiser/utils.py` file

In [4]:
#utils
import numpy as np
import psutil
import logging


def cosine_similarity(vec1, vec2):
    """
    Calculates cosine similarity between 2 vectors

    Parameters
    ----------
    vec1, vec2 : numpy array of vectorized text

    Returns
    -------
    numeric value
    """
    product_of_magnitude = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    if product_of_magnitude == 0.0:
        return 0.0
    return np.dot(vec1, vec2) / product_of_magnitude


def get_embedding(nlp, input_text):
    """
    Creates vector embeddings for input text based on nlp object

    Parameters
    ----------
    nlp : object of spacy nlp model
    input_text : text
        Provide text to be vectorized, usually skill, extracted of referenced

    Returns
    -------
    numpy array of vectorized text


    """
    doc = nlp(input_text)
    if len(doc) == 0:
        return np.zeros(300)  # Return zeros for empty texts
    return np.mean([word.vector for word in doc], axis=0)


def log_performance(function_name, start_time, end_time):
    """
    Utility function to log performance in unit of time for a function

    Parameters
    ----------
    function_name : text
        Name of the function
    start_time : time
        execution start time of the function
    end_time : time
        execution end time of the function

    """
    execution_time = end_time - start_time
    process = psutil.Process()
    cpu_percent = process.cpu_percent()
    memory_info = process.memory_info()
    memory_usage = memory_info.rss / (1024 ** 2)  # Convert to MB

    log_message = (
        f"Function: {function_name}\n"
        f"Execution time: {execution_time:.2f} seconds\n"
        f"CPU usage: {cpu_percent:.2f}%\n"
        f"Memory usage: {memory_usage:.2f} MB\n"
        "-------------------------------"
    )
    logging.info(log_message)
    print(log_message)


#### Params
- The content in the code cell below are copied from the `../laiser/params.py` file

In [ ]:
import os

# Use a placeholder if __file__ is not available
if '__file__' in locals():
    ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
else:
    ROOT_DIR = os.getcwd()  # Use current working directory as a fallback

INPUT_PATH = os.path.join(ROOT_DIR, 'input')
OUTPUT_PATH = os.path.join(ROOT_DIR, 'output')

# SKILL_DB_PATH = os.path.join(INPUT_PATH, 'combined.csv')
SKILL_DB_PATH = 'https://raw.githubusercontent.com/phanindra-max/LAiSER-datasets/master/combined.csv'

SIMILARITY_THRESHOLD = 0.85

AI_MODEL_ID = 'google/gemma-2b-it'

from google.colab import userdata
HF_TOKEN = userdata.get(HF_TOKEN)

#### LLM Methods
- The content in the code cell below are copied from the `../laiser/llm_methods.py` file

In [6]:
# llm_methods
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

torch.cuda.empty_cache()

def fetch_model_output(response):
    # Find the content between the model start tag and the last <eos> tag
    pattern = r'<start_of_turn>model\s*<eos>(.*?)<eos>\s*$'
    match = re.search(pattern, response, re.DOTALL)

    if match:
        content = match.group(1).strip()

        # Split the content by lines and filter out empty lines
        lines = [line.strip() for line in content.split('\n') if line.strip()]

        # Extract skills (lines starting with '-')
        skills = [line[1:].strip() for line in lines if line.startswith('-')]

        return skills

def get_completion_batch(queries: list, model, tokenizer, batch_size=2) -> list:
    device = "cuda:0"
    results = []

    prompt_template = """
    <start_of_turn>user
    Name all the skills present in the following description in a single list. Response should be in English and have only the skills, no other information or words. Skills should be keywords, each being no more than 3 words.
    Below text is the Description:

    {query}
    <end_of_turn>\n<start_of_turn>model
    """

    for i in range(0, len(queries), batch_size):
        batch = queries[i:i+batch_size]
        prompts = [prompt_template.format(query=query) for query in batch]

        encodeds = tokenizer(prompts, return_tensors="pt", add_special_tokens=True, padding=True, truncation=True)
        model_inputs = encodeds.to(device)

        with torch.no_grad():
            generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

        decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)

        for full_output in decoded:
            # Extract only the model's response
            response = full_output.split("<start_of_turn>model<eos>")[-1].strip()
            processed_response = fetch_model_output(response)
            results.append(processed_response)

        # Clear CUDA cache after each batch
        torch.cuda.empty_cache()

        print(f"Processed batch {i//batch_size + 1}/{(len(queries)-1)//batch_size + 1}")

    return results

def get_completion(query: str, model, tokenizer) -> str:
    device = "cuda:0"

    prompt_template = """
    <start_of_turn>user
    Name all the skills present in the following description in a single list. Response should be in English and have only the skills, no other information or words. Skills should be keywords, each being no more than 3 words.
    Below text is the Description:

    {query}
    <end_of_turn>\n<start_of_turn>model
    """
    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
    response = decoded.strip()
    processed_response = fetch_model_output(response)
    return (processed_response)

## Skill Extractor Class

- Copied from `../laiser/skill_extractor.py` file



In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

class Skill_Extractor:
    """
    Class to extract skills from text and align them to existing taxonomy
    ...

    Attributes
    ----------
    client : HuggingFace API client
    nlp : spacy nlp model

    Methods
    -------
    extract_raw(input_text: text)
        The function extracts skills from text using NER model

    align_skills(raw_skills: list, document_id='0': string):
        This function aligns the skills provided to the desired taxonomy

    extractor(data: pandas dataframe, id_column='Research ID', text_column='Text'):
        Function takes text dataset to extract and aligns skills based on available taxonomies
    ....

    """

    def __init__(self):
        self.model_id = AI_MODEL_ID
        self.nlp = spacy.load("en_core_web_lg")
        self.skill_db_df = pd.read_csv(SKILL_DB_PATH)
        self.skill_db_embeddings = np.array([get_embedding(self.nlp, label) for label in self.skill_db_df['SkillLabel']])
        if torch.cuda.is_available():
            print("GPU is available. Using GPU for Fine-tuned Language model initialization.")
            torch.cuda.empty_cache()
            self.bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16
            )
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_id,
                quantization_config=self.bnb_config,
                device_map={"": 0},
                token=HF_TOKEN,
            )
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_id, add_eos_token=True, padding_side='left', token=HF_TOKEN)
        else:
            print("GPU is not available. Using CPU for SkillNer model initialization.")
            self.ner_extractor = SkillExtractor(self.nlp, SKILL_DB, PhraseMatcher)
        return

    # Declaring a private method for extracting raw skills from input text
    def extract_raw(self, input_text):
        """
        The function extracts skills from text using Fine-Tuned Language Model's API

        Parameters
        ----------
        input_text : text
            Job advertisement / Job Description / Syllabus Description / Course Outcomes etc.

        Returns
        -------
        list: List of extracted skills from text

        Notes
        -----
        More details on which (pre-trained) language model is fine-tuned can be found in llm_methods.py
        The Function is designed only to return list of skills based on prompt passed to OpenAI's Fine-tuned model.

        """

        if torch.cuda.is_available():
            # GPU is available. Using Language model for extraction.
            extracted_skills = get_completion(input_text, self.model, self.tokenizer)
            print("Extracted_skills: ", extracted_skills)
            extracted_skills_set = set(extracted_skills)
            torch.cuda.empty_cache()
        else:
            # GPU is not available. Using SkillNer model for extraction.
            ner_extractor = self.ner_extractor
            extracted_skills_set = set()
            annotations = None
            try:
                annotations = ner_extractor.annotate(input_text)
            except ValueError as e:
                print(f"Skipping example, ValueError encountered: {e}")
            except Exception as e:
                print(f"Skipping example, An unexpected error occurred: {e}")

            for item in annotations['results']['full_matches']:
                extracted_skills_set.add(item['doc_node_value'])

            # get ngram_scored
            for item in annotations['results']['ngram_scored']:
                extracted_skills_set.add(item['doc_node_value'])

        return list(extracted_skills_set)

    def align_skills(self, raw_skills, document_id='0'):
        """
        This function aligns the skills provided to the available taxonomy

        Parameters
        ----------
        raw_skills : list
            Provide list of skill extracted from Job Descriptions / Syllabus.

        Returns
        -------
        list: List of taxonomy skills from text in JSON format
            [
                {
                    "Research ID": text_id,
                    "Skill Name": Raw skill extracted,
                    "Skill Tag": taxonomy skill tag,
                    "Correlation Coefficient": similarity_score
                },
                ...
            ]

        """
        raw_skill_embeddings = np.array([get_embedding(self.nlp, skill) for skill in raw_skills])

        # Calculate cosine similarities in bulk
        similarities = 1 - cdist(raw_skill_embeddings, self.skill_db_embeddings, metric='cosine')

        matches = []
        for i, raw_skill in enumerate(raw_skills):
            skill_matches = np.where(similarities[i] > SIMILARITY_THRESHOLD)[0]
            for match in skill_matches:
                matches.append({
                    "Research ID": document_id,
                    "Raw Skill": raw_skill,
                    "Skill Tag": self.skill_db_df.iloc[match]['SkillTag'],
                    "Correlation Coefficient": similarities[i, match]
                })

        return matches

    def extractor(self, data, id_column='Research ID', text_column='Text'):
        """
        Function takes text dataset to extract and aligns skills based on available taxonomies

        Parameters
        ----------
        data : pandas dataframe
            Dataset containing text id and actual text to extract skills.
        id_column: string
            Name of id column in the dataset. Defaults to 'Research ID'
        text_column: string
            Name of the text column in the dataset. Defaults to 'Text'

        Returns
        -------
        list: List of skill tags and similarity_score for all texts in  from text in JSON format
            [
                {
                    "Research ID": text_id
                    "Skill Name": Raw skill extracted,
                    "Skill Tag": taxonomy skill tag,
                    "Correlation Coefficient": similarity_score
                },
                ...
            ]

        """
        start_time = time.time()
        extracted = pd.DataFrame(columns=['Research ID', 'Raw Skill', 'Skill Tag', 'Correlation Coefficient'])
        for index, row in data.iterrows():
            research_id = row[id_column]
            input_text = row[text_column]
            raw_skills = self.extract_raw(input_text)
            if(len(raw_skills) == 0):
                continue
            else:
              aligned_skills = self.align_skills(raw_skills, research_id)
              extracted = extracted._append(aligned_skills, ignore_index=True)
        end_time = time.time()
        log_performance('extractor', start_time, end_time)
        return extracted

## Using the Skill Extractor

In [18]:
# Import the dataset
nlx_sample = pd.read_csv('https://raw.githubusercontent.com/phanindra-max/LAiSER-datasets/master/nlx_tx_sample_data_gwu.csv')

nlx_sample = nlx_sample[0:1]
nlx_sample = nlx_sample[['description', 'job_id']]
print("Considering", len(nlx_sample), "rows for processing...")

Considering 1 rows for processing...


In [19]:
nlx_sample

,description,job_id
0,Req ID: 29534BR\n\nPOSITION SUMMARY\n\nThis po...,69322097


In [20]:
# creating an object from the Extract class
se = Skill_Extractor() # runs __init__() method

GPU is not available. Using CPU for SkillNer model initialization.
loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [21]:
# skills output based on the taxonomy database
output = se.extractor(nlx_sample, 'job_id', 'description')

c:\Users\phani\AppData\Local\Programs\Python\Python312\Lib\site-packages\skillNer\utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


Function: extractor
Execution time: 22.05 seconds
CPU usage: 0.00%
Memory usage: 2999.23 MB
-------------------------------


C:\Users\phani\AppData\Local\Temp\ipykernel_5056\33541570.py:180: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted = extracted._append(aligned_skills, ignore_index=True)


In [22]:
# save the extracted skills to a csv file
print(output)
output.to_csv('extracted_skills_for_sample.csv', index=False)

   Research ID               Raw Skill  Skill Tag  Correlation Coefficient
0     69322097              operations   ESCO.805                 0.850277
1     69322097  business communicating   ESCO.113                 0.916486
2     69322097  business communicating   ESCO.761                 0.872133
3     69322097  business communicating   ESCO.835                 0.886956
4     69322097                 medical   ESCO.892                 0.924566
..         ...                     ...        ...                      ...
86    69322097     maintenance testing  ESCO.1278                 0.884750
87    69322097                     CSS  ESCO.1150                 1.000000
88    69322097                business   ESCO.113                 0.879549
89    69322097                business   ESCO.761                 0.889712
90    69322097                business   ESCO.835                 0.873538

[91 rows x 4 columns]
